In [1]:
import numpy as np
import osmnx as ox
import utm

from geo.math import num_haversine, vec_haversine
from geo.road import RoadNetwork, download_road_network
from geo.trajectory import load_trajectory_points

In [2]:
road_network = download_road_network("Ann Arbor, Michigan, USA")
# road_network = download_road_network("Michigan, USA")

## Matching Edges with OSMnx

We start by using OSMnx's own functions to find edge matches to locations sampled from the EVED. The first thing we need to do is to project the whole road network to the local UTM projection.

In [3]:
network_utm = ox.projection.project_graph(road_network)

The above operation converts latitudes and longitudes to a local UTM projection that works as a Cartesian plane in meters. The advantage of such projection is that you can directly use your knowledge of vectors to work out distances and other planar geometry calculations. The disadvantage is that you always need to perform the conversion before using OSMnx's functions. When converting from (_latitude_, _longitude_) to UTM, we get a converted coordinate pair (_x_, _y_) aptly named (_easting_, _northing_), along with the UTM number and letter codes. We might need these later to convert (_easting_, _northing_) back to (_latitude_, _longitude_).

In [4]:
easting, northing, zone_num, zone_ltr = utm.from_latlon(42.287702, -83.707775)

We can now call OSMnx's `nearest_edges` [function](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_edges) to determine the closest edges.

In [5]:
edge_id = ox.distance.nearest_edges(network_utm, easting, northing)

In [6]:
edge_id

(62505058, 4306881815, 0)

The first two numbers in the tuple identify the edge's nodes, and we can query it from the network using the following:

In [7]:
network_utm[edge_id[0]][edge_id[1]][0]

{'osmid': 8723817,
 'oneway': False,
 'lanes': '2',
 'highway': 'tertiary',
 'reversed': True,
 'length': 116.428,
 'speed_kph': 48.3,
 'travel_time': 8.7,
 'bearing': 267.3,
 'name': 'Glazier Way',
 'maxspeed': '30 mph'}

In [8]:
network_utm[edge_id[1]][edge_id[0]][0]

{'osmid': 8723817,
 'oneway': False,
 'lanes': '2',
 'highway': 'tertiary',
 'reversed': False,
 'length': 116.428,
 'speed_kph': 48.3,
 'travel_time': 8.7,
 'bearing': 87.3,
 'name': 'Glazier Way',
 'maxspeed': '30 mph'}

In [9]:
trajectory = load_trajectory_points(4)
latitudes = np.array([p[0] for p in trajectory])
longitudes = np.array([p[1] for p in trajectory])

In [10]:
eastings, northings, zone_num, zone_ltr = utm.from_latlon(latitudes, longitudes)

In [11]:
%%timeit
ox.distance.nearest_edges(network_utm, eastings, northings)

7.18 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
edges_0 = ox.distance.nearest_edges(network_utm, eastings, northings)

In [13]:
%%timeit
rn = RoadNetwork(road_network)
[rn.get_matching_edge(lat, lon) for lat, lon, _ in trajectory]

813 ms ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
rn = RoadNetwork(road_network)
edges_1 = [rn.get_matching_edge(lat, lon) for lat, lon, _ in trajectory]

In [15]:
e0 = [(e[0], e[1]) for e in edges_0 if e is not None]

In [16]:
e1 = [(e[0], e[1]) for e in edges_1 if e is not None]

In [17]:
len(e0), len(e1)

(203, 203)

In [18]:
max([road_network[e[0]][e[1]][0]["length"] for e in road_network.edges if "length" in road_network[e[0]][e[1]][0]])

561.48

In [19]:
ds = []
for e in road_network.edges:
    n0 = road_network.nodes[e[0]]
    n1 = road_network.nodes[e[1]]
    ds.append(num_haversine(n0['y'], n0['x'], n1['y'], n1['x']))

In [20]:
max(ds)

562.1077805301397